In [1]:
import torch
import matplotlib.pyplot as plt
from torch import optim, distributions, nn
from tqdm.autonotebook import tqdm
from sklearn.datasets import fetch_olivetti_faces
from gpzoo.kernels import NSF_RBF
from gpzoo.gp import NSF

/var/folders/dv/w3pzlt_11hdcpw6gskd45b_80000gn/T/ipykernel_18399/3667266773.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
new_model = True

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
data, _ = fetch_olivetti_faces(return_X_y=True, shuffle=True, random_state=97)

In [5]:
Y = (data)*255
Y = Y.astype(int)
Y = torch.tensor(Y)

In [6]:
idx = torch.linspace(-32, 32, 64, dtype=torch.float)
X = torch.cartesian_prod(-idx, idx).flip(dims=[1])
N = X.shape[0]

In [7]:
L =10
M = 32*32
kernel = NSF_RBF(L=L, lengthscale=2)
model = NSF(X=X, y=Y, kernel=kernel, M=M, L=L, jitter=1e-2)
idz = torch.multinomial(torch.ones(X.shape[0]), num_samples=M, replacement=False)
model.svgp.Z = nn.Parameter(X[idz])

if not new_model:
    model.load_state_dict(torch.load('eigenfaces_nsf'))
    
model.to(device)

NSF(
  (kernel): NSF_RBF()
  (svgp): SVGP(
    (kernel): NSF_RBF()
  )
)

In [8]:
X_train = X.to(device)
Y_train = Y.to(device)

In [9]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [10]:
losses = model.fit(X_train, Y_train, optimizer)

  0%|          | 0/1000 [00:00<?, ?it/s]/Users/priyankashrestha/Documents/stanford/engelhardt_research/GPzoo/gpzoo/gp.py:36: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1678402524629/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:1702.)
  L = torch.cholesky(Utilities.add_jitter(Kzz, self.jitter))
  0%|          | 3/1000 [01:04<6:01:45, 21.77s/it]